# Reading, manipulating, and writing data with Astropy

Reading and manipulating tabular data is something every astronomer has to do. Astropy supports reading and writing tables to both ASCII and binary file formats through its [unified input/output interface](http://docs.astropy.org/en/latest/io/unified.html). The core object for representing tables is the `Table`. In this tutorial, we'll learn about the class in general, then do some specific examples of reading and writing both ascii and FITS tables.

For more information about the features presented below, see the
[astropy.table](http://docs.astropy.org/en/stable/table/index.html) documentation.

In [ ]:
# Imports we'll need throughout the tutorial:
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.table import Table, QTable
from astropy.io import ascii, fits

# Astropy Tables

The astropy [Table](http://docs.astropy.org/en/stable/table/index.html) class provides an extension of NumPy structured arrays for storing and manipulating heterogeneous tables of data. [Table](http://docs.astropy.org/en/stable/table/index.html) supports many convenient features that may be familiar to users of other packages that support table data (e.g., `Pandas`). To highlight a few:

- `Table` objects can be modified in place by adding or removing columns, changing column names, or adding new rows of data.
- `Table`s support dealing with missing values.
- `Table`s support database operations like joins, concatenation, and grouping.
- `Table`s support Astropy [Units and Quantities](http://astropy.readthedocs.org/en/stable/units/index.html), especially with the `QTable` class.

## Creating an astropy `Table` from scratch


Table objects can be constructed in a variety of ways:

- By reading an existing table from a file or web URL
- By adding columns of data one by one
- By adding rows of data one by one
- From an existing data structure in memory, i.e. from a:

  - List of data columns
  - Dict of data columns
  - List of row dicts
  - Numpy homgeneous array or structured array
  - List of row records
  
See the documentation section on [Constructing a table](http://astropy.readthedocs.org/en/stable/table/construct_table.html) for the gory details and plenty of examples.

Let's create our first table object by creating an empty table and start by adding a single column:

In [ ]:
tbl = Table()
tbl['name'] = ['Graham Chapman', 'John Cleese', 'Terry Gilliam', 
               'Eric Idle', 'Terry Jones', 'Michael Palin']

In this case, the column name is given as a string index to the table object, and the data type and number of rows are inferred directly from the data. Once the table has a column, however, any new columns we add must have the same number of rows:

In [ ]:
tbl['age'] = [np.nan, 78, 77, 74, 75, 74]

# whereas, this would fail because it isn't the same length as the table:
# tbl['age'] = [77, 77]

#### Displaying or viewing tables

An instantiated table object can be viewed in a number of ways. In a Jupyter notebook, showing the table will produce a nice HTML representation of the table:

In [ ]:
tbl

If you did the same in a terminal session you get a different view that isn't as pretty but does give a bit more information about the table. In a terminal session, it would look like this:

In [ ]:
print(tbl)

We can access the table column names and data types using the `colnames` and `dtype` attributes:

In [ ]:
tbl.colnames

In [ ]:
tbl.dtype

For long tables, there is also a handy ``show_in_notebook()`` method that allows more interactive exploration like searching, and paged views of the data:

In [ ]:
tbl.show_in_notebook()

#### Accessing sections of the data

We can access the columns and rows the same as we would with numpy structured arrays: string indices access column names, and numeric indices access rows. In the examples below, notice that the returned objects are `Column`, `Row` or `Table` objects depending on the selection:

In [ ]:
tbl['name']  # access entire column

In [ ]:
tbl['name'][1] # row index 1 of column

In [ ]:
tbl[1]  # row object for row index 1

In [ ]:
tbl[1]['name']  # name column of row index 1

In [ ]:
tbl[1:3]  # select a range of rows and return a new table

In [ ]:
tbl[[1, 3]] # select specific row indices and return a new table

#### Masking rows from a table

Like Numpy arrays, we can also use boolean arrays or masks to filter tables. For example, to get the subset of rows where `'age'` is < 75 and finite:

In [ ]:
mask = (tbl['age'] < 75) & np.isfinite(tbl['age'])
tbl[mask]  # Create a new table with just the rows where the mask is True

#### Modifying a table

Once the table exists with defined columns there are a number of ways to modify the table in place.  These are fully documented in the section [Modifying a Table](http://astropy.readthedocs.org/en/stable/table/modify_table.html#modifying-a-table).

We already saw an example above of how to add new columns, but let's add another column. We can also add new rows using the [add_row()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.add_row) method:

In [ ]:
tbl['british'] = [True, True, False, True, True, True]

In [ ]:
tbl.add_row(('Eddie Izzard', 55, True))  # add a new row to the end
tbl

Notice that the `age` column really has too many output digits given that the age should be an integer.  We can change the display behavior by setting the format using a Python format string:

In [ ]:
tbl['age'].format = '%.0f'
tbl

#### Converting a table to a numpy structured array

Sometimes you may not want or be able to use a `Table` object and prefer to work with a plain numpy array.  This is easily done by passing the table to the `np.array()` constructor.  

Note that this makes a copy of the data.  If you have a huge table and don't want to waste memory, supply `copy=False` to the constructor, but be warned that changing the output numpy array will change the original table.

In [ ]:
np.array(tbl)

---

# Reading tabular data into `Table` objects

Tabular data in any format supported by the [unified input/ouput interface](http://docs.astropy.org/en/latest/io/unified.html) can be read in as a `Table` object using the [Table.read()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.read) method. Some notable examples are FITS files with a table, CSV or other delimited ASCII table, or Latex tables. Let's see some examples using files included with this repository:

In [ ]:
!ls -l example_table_data*

In [ ]:
tbl1 = Table.read('example_table_data.fits')
tbl1

The `Table.read()` method will try to figure out what type of file and data structure the input file contains:

In [ ]:
tbl2 = Table.read('example_table_data.csv')

but sometimes you have to specify other options to help it out:

In [ ]:
tbl3 = Table.read('example_table_data.tex', format='ascii.aastex')

In [ ]:
tbl4 = Table.read('example_table_data.txt', format='ascii', delimiter='|')

A number of common astronomical ascii formats are supported such as IPAC, sextractor, daophot, and CDS - see [the documentation](http://docs.astropy.org/en/latest/io/unified.html) for a full list, or the docstring:

In [ ]:
Table.read?

--- 

# Writing `Table` objects

To write a table object to disk as a binary or ascii file, use the [Table.write()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.write) method. This acts just like the `.read()` method, in that in some cases it can automatically guess the format from the filename, but may require specifying the format or other parameters (e.g., delimiter).

An example writing a FITS file:

In [ ]:
tbl1.write('test.fits', overwrite=True)

And with ASCII files of various formats:

In [ ]:
# VO table
tbl1.write('test.vot', format='votable', overwrite=True)
!head -n 30 test.vot

In [ ]:
tbl1.write('test.dat', format='ascii')

In [ ]:
tbl1.write('test.csv', format='ascii.csv')

In [ ]:
tbl1.write('test.ipac', format='ascii')

While the above is often sufficient, it's important to know that the base-line format does *not* support quite a few of the "advanced" features, like inclusion of units.  To do that you'll likely want to use the astropy table "enhanced" CSV:

In [ ]:
tbl1.write('test.ecsv', format='ascii.ecsv')

## Example: retrieving spectroscopic objects from the SDSS

In the example below, we'll query the [SDSS](http://sdss.org) spectroscopic catalog to retrieve object information for all sources within 0.5 degrees of the star HD 69497. We'll then save the table as a FITS file.

In [ ]:
from astroquery.sdss import SDSS
import astropy.coordinates as coord

In [ ]:
sc = coord.SkyCoord.from_name('HD 69497')
sc

In [ ]:
sdss_data = SDSS.query_region(sc, radius=0.5*u.deg, spectro=True)

In [ ]:
sdss_data

In [ ]:
sdss_data.write('SDSS_star.fits', overwrite=True)

---

# Table columns with units

See the [`Quantity` and Tables documentation](http://docs.astropy.org/en/latest/table/mixin_columns.html#quantity-and-qtable) for more information about this section.

Columns of normal `Table` objects can keep track of units, but do not support Quantity-like propagation of units. For example:

In [ ]:
tbl = Table()
tbl['name'] = ['Graham Chapman', 'John Cleese', 'Terry Gilliam', 
               'Eric Idle', 'Terry Jones', 'Michael Palin']
tbl['age'] = [np.nan, 78, 77, 74, 75, 74] * u.year
tbl

Notice that now the 'age' column has a unit `'yr'` listed above its data type.

While `Column` objects can store a unit, they do not act like `Quantity` objects. For example, operations with `Columns` do not propagate or update units like you'd expect:

In [ ]:
(tbl['age'] ** 2).unit

To use `Table`'s that are fully unit aware and use `Quantity` objects to store column data, instead use the `QTable` class. This class behaves similar to the `Table` and can be created in the same way:

In [ ]:
qtbl = QTable()
qtbl['name'] = ['Graham Chapman', 'John Cleese', 'Terry Gilliam', 
                'Eric Idle', 'Terry Jones', 'Michael Palin']
qtbl['age'] = [np.nan, 78, 77, 74, 75, 74] * u.year
qtbl

In [ ]:
type(qtbl['age'])

In [ ]:
(qtbl['age'] ** 2).unit

`Table` and `QTable` objects can be easily converted to the other:

In [ ]:
QTable(tbl)

In [ ]:
Table(qtbl)

---

# TODO: audit and fix exercises below

# Exercises


### Read the data

To start with, read in the two data files representing the master source list and observations source list.  The fields for the two tables are respectively documented in:

- [master_sources](http://cxc.harvard.edu/csc/columns/master.html)
- [obs_sources](http://cxc.harvard.edu/csc/columns/persrc.html)

In [ ]:
master_sources = Table.read('cdfs_master_sources.fits')
obs_sources = Table.read('cdfs_obs_sources.fits')

**`master_sources`**

Each distinct X-ray source identified on the sky is represented in the catalog by a single "master source" entry and one or more "source observation" entries, one for each observation in which the source has been detected. The master source entry records the best estimates of the properties of a source, based on the data extracted from the set of observations in which the source has been detected.  The subset of fields in our exercise table file are:

Name | Description
------ | ------------
msid  | Master source ID
name  | Source name in the Chandra catalog
ra  | Source RA (deg)
dec | Source Dec (deg)

**`obs_sources`**

The individual source entries record all of the properties about a detection extracted from a single observation, as well as associated file-based data products, which are observation-specific.  The subset of fields in our exercise table file are:

Name | Description
------ | ------------
obsid | Observation ID
obi | Observation interval
targname | Target name
gti_obs | Observation date
flux_aper_b | Broad band (0.5 - 7 keV) flux (erg/cm2/sec)
src_cnts_aper_b | Broad band source counts
ra_b | Source RA (deg)
dec_b | Source Dec (deg)
livetime | Observation duration (sec)
posid | Position ID
theta | Off-axis angle (arcmin)
msid | Master source ID

### Exploring the data
Do the following to explore the two tables:

- Display the data for each table in IPython notebook using the normal way of showing the value of a variable.
- Get a list of the column names for each table.  *Hint*: use `<TAB>` completion to easily discover all the attributes and methods, e.g. type `master_sources.` and then hit the `<TAB>` key.
- Find the length of each table.
- Find the column datatypes for each table.

Normally one displays a table in IPython notebook by entering the variable name in a cell and pressing `shift-Enter`.  In a terminal session the default method is using something like `print(my_table)`.  In both cases the `Table` object prefers to display only a screenful of data to prevent having a zillion lines of output if the table is huge.  If you really want to see all the data you can use the [Table.pprint](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.pprint) method. If you are using a Jupyter notebook interface, try the `show_in_notebook()` method.

- Display all the rows of the `master_sources` table using its `pprint()` method.
- If you are working in a regular terminal window (not IPython notebook), try the `more()` method as well.

### Modifying tables
For our analysis we don't actually need the `obi` (observation interval) column in the `obs_sources` table.

- Remove the `obi` column from the `obs_sources` table.

The `gti_obs` column name is a bit obscure (GTI is a good time interval, FWIW).

- Rename the `gti_obs` column to `obs_date`.

It would be nice to have a count rate in addition to the source counts.

- Add a new column `src_rate_aper_b` which is the source counts divided by observation duration in sec.

Some of the sources have a negative net flux in the broad band

### Looking at the observation source data
For each source detected in an individual observation (in the `obs_sources` table), let's look at the source flux values.

- Use the matplotlib [`hist()`]( http://matplotlib.org/api/pyplot_api.html?highlight=pyplot.hist#matplotlib.pyplot.hist) function to make a histogram of the source fluxes.  Since the fluxes vary by orders of magnitude,
  use the `numpy.log10` to put the fluxes in log space.

- Also make the same plot but using only sources within 4 arcmin of the center.  *HINT*: use a boolean mask to select values of `theta` that are less than 4.0.